In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import pandas as pd
import xarray as xr

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
from ecephys_analyses.data import load
import ecephys.plot as eplt

In [6]:
from scipy.stats import mode

def estimate_fs(da):
    sample_period = mode(np.diff(da.time.values)).mode[0]
    assert isinstance(sample_period, np.timedelta64)
    sample_period = sample_period / pd.to_timedelta(1, 's')
    return 1 / sample_period

In [7]:
from ripple_detection.core import gaussian_smooth

def get_smoothed_da(da, smoothing_sigma=10, in_place=False):
    if not in_place:
        da = da.copy()
    da.values = gaussian_smooth(da, smoothing_sigma, estimate_fs(da))
    return da    

def get_smoothed_ds(ds, smoothing_sigma=10, in_place=False):
    if not in_place:
        ds = ds.copy()
    for da_name, da in ds.items():
        ds[da_name] = get_smoothed_da(da, smoothing_sigma, in_place)
    return ds

In [8]:
from ecephys.scoring import get_timepoint_states

def filter_states(ds, hypnogram, states):
    timepoint_states = get_timepoint_states(hypnogram, ds.time)
    return ds.where(np.isin(timepoint_states, states))

In [9]:
def load_condition_data(subject, experiment, condition, power_as_percentage_of=None):
    hypnogram = load.load_hypnogram(subject, experiment, condition)    
    spws = load.load_spws(subject, experiment, condition, condition_start_dt=hypnogram.start_time.min())
    bp = load.load_bandpower(subject, experiment, condition)
    if power_as_percentage_of:
        bp = bp / power_as_percentage_of * 100
    
    bp = get_smoothed_ds(bp, smoothing_sigma=10, in_place=True)
    
    return hypnogram, spws, bp

In [10]:
def plot_condition(hypnogram, spws, power):

    fig, (mpta_swa_ax, sr_swa_ax, density_ax, duration_ax, amplitude_ax, integral_ax) = plt.subplots(6, 1, figsize=(22,12))
    xmargin=0
    
    mpta_swa_ylabel = 'CTX delta, WM ref'
    sr_swa_ylabel = 'SR delta, WM ref'
    mpta_swa_ylabel = mpta_swa_ylabel + ', \n % BSL NREM mean'
    sr_swa_ylabel = sr_swa_ylabel + ', \n % BSL NREM mean'

    # units = (uV^2/Hz)
    sns.lineplot(x=power.time.values, y=power.mpta_delta_wm_ref.values, color='black', ax=mpta_swa_ax)
    mpta_swa_ax.set(xlabel=None, ylabel=mpta_swa_ylabel, ylim=MPTA_SWA_LIM, xmargin=xmargin, xticks=[])

    sns.lineplot(x=power.time.values, y=power.sr_delta_wm_ref.values, color='black', ax=sr_swa_ax)
    sr_swa_ax.set(xlabel=None, ylabel=sr_swa_ylabel, ylim=SR_SWA_LIM, xmargin=xmargin, xticks=[])
    
    mpta_swa_ax.axhline(100, color='r', linestyle=':')
    sr_swa_ax.axhline(100, color='r', linestyle=':')

    binwidth = 10
    nbins = np.int((spws.start_time.max() - spws.start_time.min()).total_seconds() / binwidth)
    sns.histplot(data=spws, x='start_time', bins=nbins, color='black', ax=density_ax)
    density_ax.set(xlabel=None, ylabel='Frequency (Hz)', ylim=DENSITY_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x="start_time", y="duration", s=2, ax=duration_ax)
    duration_ax.set(xlabel=None, ylabel="Duration (s)", ylim=DURATION_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x='start_time', y='sink_amplitude', s=2, ax=amplitude_ax)
    amplitude_ax.set(xlabel=None, ylabel="Sink amplitude (mA/mm)", ylim=AMPLITUDE_LIM, xmargin=xmargin, xticks=[])

    sns.scatterplot(data=spws, x='start_time', y='sink_integral', s=2, ax=integral_ax)
    integral_ax.set(xlabel="Event time (month-date hour)", ylabel="Sink integral (mA * ms)", ylim=INTEGRAL_LIM, xmargin=xmargin)

    for ax in (mpta_swa_ax, sr_swa_ax, density_ax, duration_ax, amplitude_ax, integral_ax):
        eplt.plot_hypnogram_overlay(hypnogram, ax=ax)

    plt.tight_layout()

In [11]:
def plot_both_conditions(bsl_hypnogram, bsl_spws, bsl_power, exp_hypnogram, exp_spws, exp_power):

    fig, (bsl_swa_ax, bsl_density_ax, bsl_amplitude_ax, exp_swa_ax, exp_density_ax, exp_amplitude_ax) = plt.subplots(6, 1, figsize=(22,12))

    xmargin=0
    
    bsl_swa_ylabel = 'BSL CTX delta, WM ref'
    exp_swa_ylabel = 'EXP CTX delta, WM ref'
    bsl_swa_ylabel = bsl_swa_ylabel + ', \n % BSL NREM mean'
    exp_swa_ylabel = exp_swa_ylabel + ', \n % BSL NREM mean'
    
    # Baseline
    sns.lineplot(x=bsl_power.time.values, y=bsl_power.mpta_delta_wm_ref.values, color='black', ax=bsl_swa_ax)
    bsl_swa_ax.set(xlabel=None, ylabel=bsl_swa_ylabel, ylim=MPTA_SWA_LIM, xmargin=xmargin, xticks=[])
         
    bsl_swa_ax.axhline(100, color='r', linestyle=':')

    binwidth = 10
    nbins = np.int((bsl_spws.start_time.max() - bsl_spws.start_time.min()).total_seconds() / binwidth)
    sns.histplot(data=bsl_spws, x='start_time', bins=nbins, color='black', ax=bsl_density_ax)
    bsl_density_ax.set(xlabel=None, ylabel='BSL Frequency (Hz)', ylim=DENSITY_LIM, xmargin=xmargin, xticks=[])
    
    sns.scatterplot(data=bsl_spws, x='start_time', y='sink_amplitude', s=2, ax=bsl_amplitude_ax)
    bsl_amplitude_ax.set(xlabel=None, ylabel="BSL Sink amp. (mA/mm)", ylim=AMPLITUDE_LIM, xmargin=xmargin, xticks=[])
    
    for ax in (bsl_swa_ax, bsl_density_ax, bsl_amplitude_ax):
        eplt.plot_hypnogram_overlay(bsl_hypnogram, ax=ax)

    # Experiment
    sns.lineplot(x=exp_power.time.values, y=exp_power.mpta_delta_wm_ref.values, color='black', ax=exp_swa_ax)
    exp_swa_ax.set(xlabel=None, ylabel=exp_swa_ylabel, ylim=MPTA_SWA_LIM, xmargin=xmargin, xticks=[])
        
    exp_swa_ax.axhline(100, color='r', linestyle=':')

    binwidth = 10
    nbins = np.int((exp_spws.start_time.max() - exp_spws.start_time.min()).total_seconds() / binwidth)
    sns.histplot(data=exp_spws, x='start_time', bins=nbins, color='black', ax=exp_density_ax)
    exp_density_ax.set(xlabel=None, ylabel='EXP Frequency (Hz)', ylim=DENSITY_LIM, xmargin=xmargin, xticks=[])
    
    sns.scatterplot(data=exp_spws, x='start_time', y='sink_amplitude', s=2, ax=exp_amplitude_ax)
    exp_amplitude_ax.set(xlabel=None, ylabel="EXP Sink amp. (mA/mm)", ylim=AMPLITUDE_LIM, xmargin=xmargin, xticks=[])
        
    for ax in (exp_swa_ax, exp_density_ax, exp_amplitude_ax):
        eplt.plot_hypnogram_overlay(exp_hypnogram, ax=ax)

    plt.tight_layout()

In [65]:
subject = "Segundo"
experiment = "sleep-homeostasis"
bsl_condition = "light-period-circadian-match"
exp_condition = "light-period"

# When plotting raw power
MPTA_SWA_LIM = (0, 350000)
SR_SWA_LIM = (0, 225000)

DENSITY_LIM = (0, 20)
DURATION_LIM = (0, 0.3)
AMPLITUDE_LIM = (-0.18, -0.04)
INTEGRAL_LIM = (-15, 0)

In [72]:
subject = "Valentino"
experiment = "sleep-homeostasis"
bsl_condition = "light-period-circadian-match"
exp_condition = "light-period"

# When plotting raw power
MPTA_SWA_LIM = (0, 100000)
SR_SWA_LIM = (0, 225000)

DENSITY_LIM = (0, 20)
DURATION_LIM = (0, 0.3)
AMPLITUDE_LIM = (-0.098, -0.022)
INTEGRAL_LIM = (-11, 0)

In [86]:
subject = "Doppio"
experiment = "sleep-homeostasis"
bsl_condition = "light-period-circadian-match"
exp_condition = "light-period"

# When plotting raw power
MPTA_SWA_LIM = (0, 250000)
SR_SWA_LIM = (0, 150000)

DENSITY_LIM = (0, 20)
DURATION_LIM = (0, 0.3)
AMPLITUDE_LIM = (-0.13, -0.0275)
INTEGRAL_LIM = (-13, 0)

In [93]:
subject = "Alessandro"
experiment = "sleep-homeostasis"
bsl_condition = "light-period-circadian-match"
exp_condition = "light-period"

# When plotting raw power
MPTA_SWA_LIM = (0, 175000)
SR_SWA_LIM = (0, 200000)

DENSITY_LIM = (0, 20)
DURATION_LIM = (0, 0.25)
AMPLITUDE_LIM = (-0.22, -0.06)
INTEGRAL_LIM = (-20, 0)

In [19]:
subject = "Eugene"
experiment = "sleep-homeostasis"
bsl_condition = "light-period-circadian-match"
exp_condition = "light-period"

# When plotting raw power
MPTA_SWA_LIM = (0, 225000)
SR_SWA_LIM = (0, 125000)

DENSITY_LIM = (0, 20)
DURATION_LIM = (0, 0.3)
AMPLITUDE_LIM = (-0.08, -0.015)
INTEGRAL_LIM = (-7, 0)

In [12]:
subject = "Allan"
experiment = "sleep-homeostasis"
bsl_condition = "light-period-circadian-match"
exp_condition = "light-period"

# When plotting raw power
MPTA_SWA_LIM = (0, 350000)
SR_SWA_LIM = (0, 150000)

DENSITY_LIM = (0, 20)
DURATION_LIM = (0, 0.25)
AMPLITUDE_LIM = (-0.085, -0.022)
INTEGRAL_LIM = (-8, 0)

In [20]:
# When plotting percent change
MPTA_SWA_LIM = (0, 600)
SR_SWA_LIM = (0, 300) 

In [25]:
reference_bandpower = load.load_bandpower(subject, "sleep-homeostasis", "light-period-circadian-match")
reference_hypnogram = load.load_hypnogram(subject, "sleep-homeostasis", "light-period-circadian-match")
reference = filter_states(reference_bandpower, reference_hypnogram, ["N1", "N2"]).mean()

In [26]:
(bsl_hypnogram, bsl_spws, bsl_power) = load_condition_data(subject, experiment, bsl_condition, power_as_percentage_of=reference)
(exp_hypnogram, exp_spws, exp_power) = load_condition_data(subject, experiment, exp_condition, power_as_percentage_of=reference)

In [27]:
plot_condition(bsl_hypnogram, bsl_spws, bsl_power)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
plot_condition(exp_hypnogram, exp_spws, exp_power)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [29]:
plot_both_conditions(bsl_hypnogram, bsl_spws, bsl_power, exp_hypnogram, exp_spws, exp_power)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …